First of all, you run the following code and upload your kaggle.json.

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anuvarshini98","key":"0c1621206d15c49de511192782db8fcc"}'}

Next, run the following code to set the path.

In [2]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Then, run the following to install kaggle!

In [3]:
!pip install --upgrade --force-reinstall --no-deps kaggle

Processing /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10/kaggle-1.5.10-cp36-none-any.whl
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


Run the following to add access permission to yourself to download data.

You can also see the list of datasets you can donwload!

In [4]:
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets list

In [5]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        100           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        208           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        396           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2717            True  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      20911            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       5537           False  
connectx

As you see, you can download a lot of data here. This time we just download the data in MNIST competition. Run the following to do it.

In [6]:
# !kaggle competitions download -c digit-recognizer
!kaggle competitions download -c digit-recognizer

digit-recognizer.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
import zipfile
with zipfile.ZipFile('/content/test.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [8]:
import zipfile
with zipfile.ZipFile('/content/train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

You can find the code to install data in many competitions in data description section (shown as API ...).  

The following is basically the same as what you would do in kaggle kernel, except that we need to install keras as it is not available in default in google colab.


In [9]:
!pip install -q keras
import keras

Note that you are in /content.

In [10]:
import os
print(os.getcwd())

/content


In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import further libraries
import matplotlib.pyplot as plt
import seaborn as sns

# keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ReduceLROnPlateau  
from sklearn.model_selection import train_test_split

As you are in /content, loading data can be done in the following.

In [12]:
# load training & test datasets
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

The rest is just modeling:D

In [13]:
# pandas to numpy
y_train = train["label"]
X_train = train.drop(labels=["label"], axis=1)

del train

# normalize
X_train = X_train/255.0
test = test/255.0

# reshape the data so that the data
# represents (label, img_rows, img_cols, grayscale)
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

# one-hot vector as a label (binarize the label)
y_train = to_categorical(y_train, num_classes=10)

In [14]:
# Three steps to create a CNN
# 1. Convolution
# 2. Activation
# 3. Pooling
# Repeat Steps 1,2,3 for adding more hidden layers

# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax'))

In [15]:
# compile model
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [16]:
# cross validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=1220)

In [17]:
# data argumentation
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
train_generator = gen.flow(X_train, y_train, batch_size=64)

In [18]:
# learning rate
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [19]:
batch_size=64

In [20]:
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.995):
      print("\nReached 98% accuracy so cancelling training!")
      self.model.stop_training = True

In [21]:
callbacks = myCallback()

In [22]:
# model training
model.fit_generator(train_generator, epochs=30, validation_data = (X_val, y_val), verbose=2, steps_per_epoch=len(X_train)//batch_size,
                                       callbacks=[callbacks])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
590/590 - 12s - loss: 0.1486 - accuracy: 0.9531 - val_loss: 0.0499 - val_accuracy: 0.9855
Epoch 2/30
590/590 - 10s - loss: 0.0637 - accuracy: 0.9802 - val_loss: 0.0384 - val_accuracy: 0.9890
Epoch 3/30
590/590 - 10s - loss: 0.0494 - accuracy: 0.9848 - val_loss: 0.0506 - val_accuracy: 0.9876
Epoch 4/30
590/590 - 10s - loss: 0.0441 - accuracy: 0.9866 - val_loss: 0.0529 - val_accuracy: 0.9855
Epoch 5/30
590/590 - 10s - loss: 0.0385 - accuracy: 0.9885 - val_loss: 0.0754 - val_accuracy: 0.9800
Epoch 6/30
590/590 - 10s - loss: 0.0370 - accuracy: 0.9892 - val_loss: 0.0450 - val_accuracy: 0.9900
Epoch 7/30
590/590 - 10s - loss: 0.0314 - accuracy: 0.9904 - val_loss: 0.0673 - val_accuracy: 0.9864
Epoch 8/30
590/590 - 10s - loss: 0.0306 - accuracy: 0.9909 - val_loss: 0.0649 - val_accuracy: 0.9869
Epoch 9/30
590/590 - 10s - loss: 0.0316 - accuracy: 0.9913 - val_loss: 0.0508 - val_accuracy: 0.9917
Epoch 10/30
590/590 - 10s - loss: 0.0312 - accuracy: 0.9910 - val_loss: 0.0362 - val_accura

In [26]:
# model prediction on test data
predictions = model.predict_classes(test, verbose=0)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [27]:
# make a submission file
submissions = pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
    "Label": predictions})
submissions.to_csv("my_submission.csv", index=False, header=True)

Finally by running the following command, you can submit your file to kaggle from google colab!

In [28]:
# submit the file to kaggle
!kaggle competitions submit digit-recognizer -f my_submission.csv 

100% 208k/208k [00:00<00:00, 765kB/s]
Successfully submitted to Digit Recognizer

Now you can go back to Kaggle to see where you are on the Leaderboard:D Enjoy kaggle more with google colab!